<a href="https://colab.research.google.com/github/fjoseph631/Machine-Learning/blob/master/Problem_3_HW2AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Loading the CIFAR10 data set

In [0]:
from keras.datasets import cifar10

(train_images, train_labels_original), (test_images, test_labels_original) = cifar10.load_data()

from keras.utils import to_categorical

train_labels = to_categorical(train_labels_original)
test_labels = to_categorical(test_labels_original)


In [0]:
%matplotlib inline

import matplotlib.pyplot as plt

from keras.preprocessing import image



## Randomly augmenting images

In [0]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
train_datagen = ImageDataGenerator(
    
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')
train_generator = train_datagen.flow(train_images,
    train_labels,
    batch_size=32,
   
)

validation_datagen = ImageDataGenerator()

validation_generator = validation_datagen.flow(
    test_images,
    test_labels,
    batch_size=32,
)


In [0]:
from keras import layers
from keras import models
from keras import optimizers
from keras.applications import VGG16
import numpy as np
model = models.Sequential()


## Underfit Model


In [16]:
model = models.Sequential()
# conv layers
# 1
model.add(layers.Conv2D(2, (3, 3), activation='relu',input_shape=(32,32, 3,)))
model.add(layers.MaxPooling2D(2, 2))

# dense layers
model.add(layers.Flatten())

model.add(layers.Dense(1, activation='sigmoid'))
model.compile(
    loss='binary_crossentropy', 
    optimizer='rmsprop', 
    metrics=['acc'])
model.summary()
epochs =6
history = model.fit(train_images,
                    train_labels_original,
                    epochs=epochs, 
                    batch_size=32, 
                    validation_data=(test_images, test_labels_original))

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 30, 30, 2)         56        
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 15, 15, 2)         0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 450)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 451       
Total params: 507
Trainable params: 507
Non-trainable params: 0
_________________________________________________________________
Train on 50000 samples, validate on 10000 samples
Epoch 1/6
50000/50000 [==============================] - 9s 181us/step - loss: -55.7929 - acc: 0.1000 - val_loss: -55.7983 - val_acc: 0.1000
Epoch 2/6
50000/50000 [==============================] - 9s 185


## Overfit Model training on regular data



In [17]:
model = models.Sequential()
# conv layers
# 1
model.add(layers.Conv2D(8, (3, 3), activation='relu',input_shape=(32,32, 3,)))
model.add(layers.MaxPooling2D(2, 2))
# 3
model.add(layers.Conv2D(16, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(2, 2))
# 4
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(2, 2))

# dense layers
model.add(layers.Flatten())
# DROPOUT
#model.add(layers.Dropout(0.1))
model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
model.compile(
    loss='binary_crossentropy', 
    optimizer='rmsprop', 
    metrics=['acc'])
model.summary()

epochs = 10
history = model.fit(train_images,
                    train_labels,
                    epochs=epochs, 
                    batch_size=32, 
                    validation_data=(test_images, test_labels))

24576/50000 [=============>................] - ETA: 6s - loss: 2.8887 - acc: 0.8198

KeyboardInterrupt: ignored

##Proper Fit Model

In [0]:
model = models.Sequential()
# conv layers
# 1
model.add(layers.Conv2D(8, (3, 3), activation='relu',input_shape=(32,32, 3,)))
model.add(layers.MaxPooling2D(2, 2))
# 3
model.add(layers.Conv2D(16, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(2, 2))
# 4
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(2, 2))
# dense layers
model.add(layers.Flatten())
# DROPOUT
model.add(layers.Dropout(0.25))
model.add(layers.Dense(2048, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
model.compile(
    loss='binary_crossentropy', 
    optimizer='rmsprop', 
    metrics=['acc'])
model.summary()
history = model.fit_generator(
    train_generator,
    steps_per_epoch=1000,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=500
)
epochs=6
"""history = model.fit(train_images,
                train_labels,
                epochs=epochs, 
                batch_size=32, 
                validation_data=(test_images, test_labels))"""

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 30, 30, 8)         224       
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 15, 15, 8)         0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 13, 13, 16)        1168      
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 6, 6, 16)          0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 4, 4, 32)          4640      
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 2, 2, 32)          0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 128)             

## Display curves of loss and accuracy during training

In [0]:
acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

# training and validation accuracy

plt.plot(epochs, acc, 'bo', label='training acc')
plt.plot(epochs, val_acc, 'b', label='validation acc')
plt.title('training and validation accuracy')
plt.legend()

plt.figure()

# training and validation loss

plt.plot(epochs, loss, 'bo', label='training loss')
plt.plot(epochs, val_loss, 'b', label='validation loss')
plt.title('training and validation loss')
plt.legend()

plt.show()